# VectorStore: Chroma and Sentence Transformer (all-MiniLM-L6-v2) with  Basic Examples

This notebook demonstrates how to use the `ChromaVectorStore` in `Floki` for storing, querying, and filtering documents. We will explore:

* Initializing the `SentenceTransformerEmbedder` embedding function and `ChromaVectorStore`.
* Adding documents with text and metadata.
* Retrieving documents by ID.
* Updating documents.
* Deleting documents.
* Performing similarity searches.
* Filtering results based on metadata.

## Install Required Libraries
Before starting, ensure the required libraries are installed:

In [ ]:
!pip install floki-ai python-dotenv chromadb sentence-transformers

## Enable Logging

In [1]:
import logging

logging.basicConfig(level=logging.INFO)

## Load Environment Variables

Load API keys or other configuration values from your `.env` file using `dotenv`.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Initializing SentenceTransformer Embedding Function

The default embedding function is `SentenceTransformerEmbedder`, but we will initialize it explicitly for clarity.

In [3]:
from floki.document.embedder import SentenceTransformerEmbedder

embedding_function = SentenceTransformerEmbedder(
    model="all-MiniLM-L6-v2",
    cache_dir="model"
)

INFO:datasets:PyTorch version 2.5.1 available.
INFO:floki.document.embedder.sentence:Loading SentenceTransformer model from local path: model
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: model
INFO:floki.document.embedder.sentence:Model loaded successfully.


## Initializing the ChromaVectorStore

To start, create an instance of the `ChromaVectorStore` and set the `embedding_function` to the instance of `SentenceTransformerEmbedder`

In [4]:
from floki.storage import ChromaVectorStore

# Initialize ChromaVectorStore
store = ChromaVectorStore(
    name="example_collection",  # Name of the collection
    embedding_function=embedding_function,
    persistent=False,           # No persistence for this example
    host="localhost",           # Host for the Chroma server
    port=8000                   # Port for the Chroma server
)

INFO:floki.storage.vectorstores.chroma:ChromaVectorStore initialized with collection: example_collection


## Adding Documents
We will use Document objects to add content to the collection. Each Document includes text and optional metadata.

### Creating Documents

In [5]:
from floki.types.document import Document

# Example Lord of the Rings-inspired conversations
documents = [
    Document(
        text="Gandalf: A wizard is never late, Frodo Baggins. Nor is he early; he arrives precisely when he means to.",
        metadata={"topic": "wisdom", "location": "The Shire"}
    ),
    Document(
        text="Frodo: I wish the Ring had never come to me. I wish none of this had happened.",
        metadata={"topic": "destiny", "location": "Moria"}
    ),
    Document(
        text="Aragorn: You cannot wield it! None of us can. The One Ring answers to Sauron alone. It has no other master.",
        metadata={"topic": "power", "location": "Rivendell"}
    ),
    Document(
        text="Sam: I can't carry it for you, but I can carry you!",
        metadata={"topic": "friendship", "location": "Mount Doom"}
    ),
    Document(
        text="Legolas: A red sun rises. Blood has been spilled this night.",
        metadata={"topic": "war", "location": "Rohan"}
    ),
    Document(
        text="Gimli: Certainty of death. Small chance of success. What are we waiting for?",
        metadata={"topic": "bravery", "location": "Helm's Deep"}
    ),
    Document(
        text="Boromir: One does not simply walk into Mordor.",
        metadata={"topic": "impossible tasks", "location": "Rivendell"}
    ),
    Document(
        text="Galadriel: Even the smallest person can change the course of the future.",
        metadata={"topic": "hope", "location": "Lothlórien"}
    ),
    Document(
        text="Théoden: So it begins.",
        metadata={"topic": "battle", "location": "Helm's Deep"}
    ),
    Document(
        text="Elrond: The strength of the Ring-bearer is failing. In his heart, Frodo begins to understand. The quest will claim his life.",
        metadata={"topic": "sacrifice", "location": "Rivendell"}
    )
]

### Adding Documents to the Collection

In [6]:
store.add_documents(documents=documents)
print(f"Number of documents in the collection: {store.count()}")

INFO:floki.document.embedder.sentence:Generating embeddings for 10 input(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Number of documents in the collection: 10


## Retrieving Documents

Retrieve documents by their IDs or fetch all items in the collection.

In [7]:
# Retrieve all documents
retrieved_docs = store.get()
print("Retrieved documents:")
for doc in retrieved_docs:
    print(f"ID: {doc['id']}, Text: {doc['document']}, Metadata: {doc['metadata']}")

Retrieved documents:
ID: b70624bd-e2cd-45c1-91ea-5793e7ca379b, Text: Gandalf: A wizard is never late, Frodo Baggins. Nor is he early; he arrives precisely when he means to., Metadata: {'location': 'The Shire', 'topic': 'wisdom'}
ID: 9138873e-19a8-4261-bb2a-4dc7cf88160a, Text: Frodo: I wish the Ring had never come to me. I wish none of this had happened., Metadata: {'location': 'Moria', 'topic': 'destiny'}
ID: 97f0faca-c592-4464-8caf-35a8bf334250, Text: Aragorn: You cannot wield it! None of us can. The One Ring answers to Sauron alone. It has no other master., Metadata: {'location': 'Rivendell', 'topic': 'power'}
ID: e953a8ad-e73d-41bb-9fc6-275e0abd3c71, Text: Sam: I can't carry it for you, but I can carry you!, Metadata: {'location': 'Mount Doom', 'topic': 'friendship'}
ID: 7698118d-33b7-4d63-8fc8-c81ef7514d29, Text: Legolas: A red sun rises. Blood has been spilled this night., Metadata: {'location': 'Rohan', 'topic': 'war'}
ID: 28a85d66-d0ce-4cc0-a60d-2d7978b6b337, Text: Gimli: Certai

## Updating Documents

You can update existing documents' text or metadata using their IDs.

In [8]:
# Retrieve a document by its ID
retrieved_docs = store.get()  # Get all documents to find the ID
doc_id = retrieved_docs[0]['id']  # Select the first document's ID for this example

# Define updated text and metadata
updated_text = "Gandalf: Even the wisest cannot foresee all ends, but hope remains while the Company is true."
updated_metadata = {"topic": "hope and wisdom", "location": "Fangorn Forest"}

# Update the document's text and metadata in the store
store.update(ids=[doc_id], documents=[updated_text], metadatas=[updated_metadata])

# Verify the update
updated_doc = store.get(ids=[doc_id])
print(f"Updated document: {updated_doc}")

INFO:floki.document.embedder.sentence:Generating embeddings for 1 input(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Updated document: [{'id': 'b70624bd-e2cd-45c1-91ea-5793e7ca379b', 'metadata': {'location': 'Fangorn Forest', 'topic': 'hope and wisdom'}, 'document': 'Gandalf: Even the wisest cannot foresee all ends, but hope remains while the Company is true.'}]


## Deleting Documents

Delete documents by their IDs.

In [9]:
# Delete a document by ID
doc_id_to_delete = retrieved_docs[2]['id']
store.delete(ids=[doc_id_to_delete])

# Verify deletion
print(f"Number of documents after deletion: {store.count()}")

Number of documents after deletion: 9


## Similarity Search

Perform a similarity search using text queries. The embedding function automatically generates embeddings for the input query.

In [10]:
# Search for similar documents based on a query
query = "wise advice"
results = store.search_similar(query_texts=query, k=2)

# Display results
print("Similarity search results:")
for doc, metadata in zip(results["documents"], results["metadatas"]):
    print(f"Text: {doc}")
    print(f"Metadata: {metadata}")

INFO:floki.document.embedder.sentence:Generating embeddings for 1 input(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity search results:
Text: ['Gandalf: Even the wisest cannot foresee all ends, but hope remains while the Company is true.', 'Gimli: Certainty of death. Small chance of success. What are we waiting for?']
Metadata: [{'location': 'Fangorn Forest', 'topic': 'hope and wisdom'}, {'location': "Helm's Deep", 'topic': 'bravery'}]


## Filtering Results

Filter results based on metadata.

In [11]:
# Search for documents with specific metadata filters
filter_conditions = {
    "$and": [
        {"location": {"$eq": "Fangorn Forest"}},
        {"topic": {"$eq": "hope and wisdom"}}
    ]
}

filtered_results = store.query_with_filters(query_texts=["journey"], where=filter_conditions, k=3)

INFO:floki.document.embedder.sentence:Generating embeddings for 1 input(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
filtered_results

{'ids': [['b70624bd-e2cd-45c1-91ea-5793e7ca379b']],
 'embeddings': None,
 'documents': [['Gandalf: Even the wisest cannot foresee all ends, but hope remains while the Company is true.']],
 'uris': None,
 'data': None,
 'metadatas': [[{'location': 'Fangorn Forest', 'topic': 'hope and wisdom'}]],
 'distances': [[0.7907481789588928]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## Resetting the Database

Reset the database to clear all stored data.

In [13]:
store.client.list_collections()

[Collection(name=example_collection)]

In [14]:
# Reset the collection
store.reset()

In [15]:
store.client.list_collections()

[]